In [3]:
%load_ext autoreload
%autoreload 2
import penzai
from penzai import pz
pz.ts.register_as_default()
pz.ts.register_autovisualize_magic()
pz.enable_interactive_context()

In [4]:
from micrlhf.llama import LlamaTransformer
llama = LlamaTransformer.from_pretrained("models/gemma-2b.gguf", device_map="auto",
                                         from_type="gemma", load_eager=True)

embed.embeddings [2048, 256000] {'vocabulary': 256000, 'embedding': 2048} q8_0
blocks.0.pre_attn_norm.scale.weights [2048] {'embedding': 2048} fp32
blocks.0.attn.query.weights [2048, 2048] {'embedding': 2048, 'kv_heads': 1, 'q_rep': 8, 'projection': 256} q8_0
blocks.0.attn.key.weights [2048, 256] {'embedding': 2048, 'kv_heads': 1, 'projection': 256} q8_0
blocks.0.attn.value.weights [2048, 256] {'embedding': 2048, 'kv_heads': 1, 'projection': 256} q8_0
blocks.0.attn.output.weights [2048, 2048] {'kv_heads': 1, 'q_rep': 8, 'projection': 256, 'embedding': 2048} q8_0
blocks.0.pre_mlp_norm.scale.weights [2048] {'embedding': 2048} fp32
blocks.0.mlp.gate_proj.weights [2048, 16384] {'embedding': 2048, 'neurons': 16384} q8_0
blocks.0.mlp.up_proj.weights [2048, 16384] {'embedding': 2048, 'neurons': 16384} q8_0
blocks.0.mlp.out_proj.weights [16384, 2048] {'neurons': 16384, 'embedding': 2048} q8_0
blocks.1.pre_attn_norm.scale.weights [2048] {'embedding': 2048} fp32
blocks.1.attn.query.weights [2048

In [1]:
from transformers import AutoTokenizer
tokenizer = AutoTokenizer.from_pretrained("alpindale/gemma-2b")
tokenizer.padding_side = "right"

/home/neverix/.cache/pypoetry/virtualenvs/micrlhf-progress-a058ydGG-py3.12/lib/python3.12/site-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


In [2]:
from transformers import AutoModelForCausalLM
model = AutoModelForCausalLM.from_pretrained("alpindale/gemma-2b")

Gemma's activation function should be approximate GeLU and not exact GeLU.
Changing the activation function to `gelu_pytorch_tanh`.if you want to use the legacy `gelu`, edit the `model.config` to set `hidden_activation=gelu`   instead of `hidden_act`. See https://github.com/huggingface/transformers/pull/29402 for more details.


Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

/home/neverix/.cache/pypoetry/virtualenvs/micrlhf-progress-a058ydGG-py3.12/lib/python3.12/site-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


In [26]:
tokens = tokenizer.encode("The quick brown fox jumps over the lazy dog")

In [29]:
import torch
from collections import OrderedDict
for m in model.modules():
    m._forward_hooks = OrderedDict()
model.model.layers[0].register_forward_hook(lambda self, input, output: print(input[0]))
model(torch.LongTensor(tokens).unsqueeze(0)).logits

torch.Size([1, 10, 2048]) tensor([[[ 5.3917,  0.8673, -1.4363,  ..., -0.2969,  0.0157, -0.4033],
         [10.3856, -2.8726, -3.0936,  ..., -0.2983,  1.9003, -3.1157],
         [13.4350, -2.3754,  0.7568,  ..., -0.0687,  2.4528, -1.1214],
         ...,
         [10.5182, -1.1270, -4.5299,  ..., -0.3439,  2.0329,  0.9723],
         [15.4680, -1.9224,  1.9666,  ...,  1.8672,  2.1324, -2.6296],
         [12.1092, -0.8121,  2.5301,  ..., -1.1656,  1.1546, -0.0511]]],
       grad_fn=<MulBackward0>)


tensor([[[-26.5950, -13.8833, -18.2747,  ..., -21.2892, -23.2392, -26.5799],
         [-42.4406, -33.3739, -47.8576,  ..., -42.1095, -46.7014, -42.5487],
         [-42.5846, -30.0222, -34.0748,  ..., -40.6030, -38.9625, -42.6937],
         ...,
         [-41.7852, -19.4810, -38.3410,  ..., -38.2935, -41.6302, -41.8573],
         [-39.3097, -18.4143, -34.3758,  ..., -36.4977, -40.6678, -39.3800],
         [-42.2921, -23.0559, -36.2486,  ..., -38.2521, -37.0666, -42.3374]]],
       grad_fn=<UnsafeViewBackward0>)

In [30]:
import torch
model.model.embed_tokens(torch.LongTensor(tokens).unsqueeze(0))

tensor([[[ 0.1191,  0.0192, -0.0317,  ..., -0.0066,  0.0003, -0.0089],
         [ 0.2295, -0.0635, -0.0684,  ..., -0.0066,  0.0420, -0.0688],
         [ 0.2969, -0.0525,  0.0167,  ..., -0.0015,  0.0542, -0.0248],
         ...,
         [ 0.2324, -0.0249, -0.1001,  ..., -0.0076,  0.0449,  0.0215],
         [ 0.3418, -0.0425,  0.0435,  ...,  0.0413,  0.0471, -0.0581],
         [ 0.2676, -0.0179,  0.0559,  ..., -0.0258,  0.0255, -0.0011]]],
       grad_fn=<EmbeddingBackward0>)

In [36]:
llama.select().at_instances_of(pz.nn.EmbeddingLookup).get_sequence()[0](inputs.tokens).unwrap("batch", "seq", "embedding")[0]

# jax.Array bfloat16(10, 2048) ≈0.00042 ±0.16 [≥-1e+01, ≤6.3] zero:276 nonzero:20_204
  Array([[0.119141, 0.0187988, -0.0319824, ..., -0.00628662, 0,
          -0.00628662],
         [0.229492, -0.0639648, -0.0683594, ..., -0.00872803, 0.0437012,
          -0.0698242],
         [0.296875, -0.0512695, 0.0163574, ..., 0, 0.052002, -0.026001],
         ...,
         [0.232422, -0.0256348, -0.100586, ..., -0.00540161, 0.0432129,
          0.0216064],
         [0.341797, -0.0429688, 0.0429688, ..., 0.0424805, 0.0478516,
          -0.0583496],
         [0.267578, -0.0189209, 0.0568848, ..., -0.0263672, 0.0263672, 0]], dtype=bfloat16)

In [33]:
import jax.numpy as jnp
import jax
from micrlhf.llama import LlamaBlock
get_resids = llama.select().at_instances_of(LlamaBlock).apply_with_selected_index(lambda i, x:
    pz.nn.Sequential([
        pz.de.TellIntermediate.from_config(tag=f"resid_pre_{i}"),
        x
    ])
)
get_resids = pz.de.CollectingSideOutputs.handling(get_resids, tag_predicate=lambda x: x.startswith("resid_pre"))
token_array = jnp.asarray([tokens] * 4)
token_array = jax.device_put(token_array, jax.sharding.NamedSharding(llama.mesh, jax.sharding.PartitionSpec("dp", "sp")))
token_array = pz.nx.wrap(token_array, "batch", "seq")
inputs = llama.inputs.from_basic_segments(token_array)
get_resids(inputs)[1][0].value.unwrap("batch", "seq", "embedding")[0]

# jax.Array bfloat16(10, 2048) ≈0.00042 ±0.16 [≥-1e+01, ≤6.3] zero:276 nonzero:20_204
  Array([[0.119141, 0.0187988, -0.0319824, ..., -0.00628662, 0,
          -0.00628662],
         [0.229492, -0.0639648, -0.0683594, ..., -0.00872803, 0.0437012,
          -0.0698242],
         [0.296875, -0.0512695, 0.0163574, ..., 0, 0.052002, -0.026001],
         ...,
         [0.232422, -0.0256348, -0.100586, ..., -0.00540161, 0.0432129,
          0.0216064],
         [0.341797, -0.0429688, 0.0429688, ..., 0.0424805, 0.0478516,
          -0.0583496],
         [0.267578, -0.0189209, 0.0568848, ..., -0.0263672, 0.0263672, 0]], dtype=bfloat16)

In [8]:
from micrlhf.sampling import sample
texts = sample(llama, tokenizer, "Write me a poem about Machine Learning.",
               batch_size=4, do_sample=True)
texts

  0%|          | 0/64 [00:00<?, ?it/s]

KeyboardInterrupt: 